In [32]:
import nltk
import polars as pl
import time
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/legume/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
df = pl.read_csv('./twitter_dataset.csv')
df

Tweet_ID,Username,Text,Retweets,Likes,Timestamp
i64,str,str,i64,i64,str
1,"""julie81""","""Party least receive say or sin…",2,25,"""2023-01-30 11:00:51"""
2,"""richardhester""","""Hotel still Congress may membe…",35,29,"""2023-01-02 22:45:58"""
3,"""williamsjoseph""","""Nice be her debate industry th…",51,25,"""2023-01-18 11:25:19"""
4,"""danielsmary""","""Laugh explain situation career…",37,18,"""2023-04-10 22:06:29"""
5,"""carlwarren""","""Involve sense former often app…",27,80,"""2023-01-24 07:12:21"""
…,…,…,…,…,…
9996,"""ntate""","""Agree reflect military box abi…",81,86,"""2023-01-15 11:46:20"""
9997,"""garrisonjoshua""","""Born which push still. Degree …",73,100,"""2023-05-06 00:46:54"""
9998,"""adriennejackson""","""You day agent likely region. T…",10,62,"""2023-02-27 14:55:08"""


In [48]:
t0 = time.time()
words_ordered_df = (
    df.lazy().with_columns(
        pl.col('Text').str.to_lowercase()
            .str.extract_all('\w+')
            .list.unique()
            .alias('words')
    )
        .select(['Likes', 'words'])
        .explode('words')
        .filter(~pl.col('words').is_in(stop_words))
        .group_by('words')
        .agg(pl.sum('Likes').alias('likes'))
        .sort('likes', descending=True)
        .collect()
)

print(f'elapsed: {time.time() - t0:.3f} s')
words_ordered_df.head(100).write_csv('top_words.csv')

elapsed: 0.032 s


In [49]:
words_ordered_df.head(10)

words,likes
str,i64
"""hard""",20628
"""maybe""",20209
"""man""",20150
"""exactly""",20056
"""everyone""",19958
"""boy""",19845
"""teach""",19845
"""tax""",19713
"""instead""",19706
